# Spesifikasi

MP = 5

Electrolytic Capacitor: 25 V Nippon KZE series

# Single Capacitor

47	100	220	330	470	680	820	1000	1500	1800	2200	2700

# Series

In [1]:
import csv
from itertools import combinations

def read_capacitor_values_from_csv(csv_filename):
    capacitors = []

    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            capacitor = {
                'value': float(row['Capacitor']),
                'volume': float(row['Volume'])  # Add this line to read the volume
            }
            capacitors.append(capacitor)

    return capacitors

def calculate_series_combinations(capacitors, max_parallel=5):
    series_combinations = []

    for r in range(1, min(max_parallel + 1, len(capacitors) + 1)):
        # Generate all possible combinations of r capacitors
        for combination in combinations(capacitors, r):
            # Calculate equivalent capacitance for the series combination
            equivalent_capacitance = sum(1 / capacitor['value'] for capacitor in combination) ** -1

            # Calculate total volume for the series combination
            total_volume = sum(capacitor['volume'] for capacitor in combination)

            # Append the combination, equivalent capacitance, and total volume to the list
            series_combinations.append({
                'combination': [capacitor['value'] for capacitor in combination],
                'equivalent_capacitance': equivalent_capacitance,
                'total_volume': total_volume
            })

    return series_combinations

def save_to_csv(series_combinations, csv_filename):
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Combination', 'Capacitance', 'Total Volume']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for combination in series_combinations:
            writer.writerow({
                'Combination': str(combination['combination']),
                'Capacitance': round(combination['equivalent_capacitance']),
                'Total Volume': round(combination['total_volume'], 2)
            })

# Specify the CSV file containing capacitor values and volumes
capacitor_csv_filename = 'capacitor.csv'

# Read capacitor values and volumes from CSV
capacitors = read_capacitor_values_from_csv(capacitor_csv_filename)

# Maximum parallel connections (MP)
max_parallel = 5

# Calculate series combinations
series_combinations = calculate_series_combinations(capacitors, max_parallel)

# Save to CSV file
csv_filename = 'series_combinations.csv'
save_to_csv(series_combinations, csv_filename)
print(f"Series combinations saved to {csv_filename}")


Series combinations saved to series_combinations.csv


In [ ]:
import csv
from itertools import combinations

def read_capacitor_values_from_csv(csv_filename):
    capacitors = []

    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            capacitor = {
                'value': float(row['Capacitor']),
                'volume': float(row['Volume'])  # Add this line to read the volume
            }
            capacitors.append(capacitor)

    return capacitors

def calculate_parallel_combinations(capacitors, max_parallel=5, max_equivalent_capacitance=1000):
    parallel_combinations = []

    for r in range(1, min(max_parallel + 1, len(capacitors) + 1)):
        # Generate all possible combinations of r capacitors
        for combination in combinations(capacitors, r):
            # Calculate equivalent capacitance for the parallel combination
            equivalent_capacitance = sum(capacitor['value'] for capacitor in combination)

            # Calculate total volume for the parallel combination
            total_volume = sum(capacitor['volume'] for capacitor in combination)

            # Check if the equivalent capacitance is below the threshold
            if equivalent_capacitance <= max_equivalent_capacitance:
                # Append the combination, equivalent capacitance, and total volume to the list
                parallel_combinations.append({
                    'combination': [capacitor['value'] for capacitor in combination],
                    'equivalent_capacitance': equivalent_capacitance,
                    'total_volume': total_volume
                })

    return parallel_combinations

def save_to_csv(parallel_combinations, csv_filename):
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Combination', 'Capacitance', 'Total Volume']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for combination in parallel_combinations:
            writer.writerow({
                'Combination': str(combination['combination']),
                'Capacitance': round(combination['equivalent_capacitance']),
                'Total Volume': round(combination['total_volume'], 2)
            })

# Specify the CSV file containing capacitor values and volumes
capacitor_csv_filename = 'capacitor.csv'

# Read capacitor values and volumes from CSV
capacitors = read_capacitor_values_from_csv(capacitor_csv_filename)

# Maximum parallel connections
max_parallel = 5

# Maximum equivalent capacitance threshold
max_equivalent_capacitance = 1000

# Calculate parallel combinations
parallel_combinations = calculate_parallel_combinations(capacitors, max_parallel, max_equivalent_capacitance)

# Save to CSV file
csv_filename = 'parallel_combinations.csv'
save_to_csv(parallel_combinations, csv_filename)
print(f"Parallel combinations saved to {csv_filename}")


In [2]:
import csv

def read_csv(file_path):
    data = []
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)
        for row in reader:
            data.append(row)
    return headers, data

def write_csv(file_path, headers, data):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        writer.writerows(data)

def combine_files(file1_path, file2_path, output_file_path):
    headers_file1, data_file1 = read_csv(file1_path)
    headers_file2, data_file2 = read_csv(file2_path)

    # Ensure headers are the same for both files
    if headers_file1 != headers_file2:
        raise ValueError("Headers in the two files are not the same.")

    # Create a list to store the combined data
    combined_data = []

    # Combine data
    for row_file1 in data_file1:
        for row_file2 in data_file2:
            combination = f"{row_file1[0]} and {row_file2[0]}"
            capacitance_sum = float(row_file1[1]) + float(row_file2[1])
            total_volume_sum = round(float(row_file1[2]) + float(row_file2[2]), 2)

            # Discard rows with Total Capacitance more than 1000
            if capacitance_sum <= 1000:
                combined_data.append([combination, capacitance_sum, total_volume_sum])

    # Write the combined data to a new CSV file
    write_csv(output_file_path, ['Combination', 'Capacitance', 'Total Volume'], combined_data)

# Example usage
combine_files('parallel_combinations.csv', 'series_combinations.csv', 'combined_result.csv')
